# Depression-Detection-LT-EDI-ACL-2022

In [1]:
import pandas as pd
import numpy as np
from datatable import (dt, f, by, ifelse, update, sort, count, min, max, mean, sum, rowsum)
import re
import string
import math
import demoji
import emoji

In [2]:
train = dt.fread('train.tsv')
valid = dt.fread('dev_with_labels.tsv')
train.head()

,PID,Text_data,Label
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,train_pid_1,Waiting for my mind to have a breakdown once the “…,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass int…,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will be t…,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my engl…",moderate
4,train_pid_5,Sat in the dark and cried myself going into the ne…,moderate
5,train_pid_6,I will probably end it when my mum isn't around an…,moderate
6,train_pid_7,Fuck 2019 : Left abusive relationship. Moved in wi…,moderate
7,train_pid_8,I am at a new year's eve party and I want to cry. …,moderate
8,train_pid_9,Death of my father : My father died in the beginni…,moderate


In [3]:
train.shape

(8891, 3)

In [4]:
valid.shape

(4496, 3)

In [5]:
train.nunique()

,PID,Text_data,Label
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,8891,2720,3


沒有缺值

In [6]:
df_train = train.to_pandas()
df_valid = valid.to_pandas()
df_train.isnull().sum()
df_valid.isnull().sum()

PID          0
Text_data    0
Label        0
dtype: int64

### Preprocessing

In [7]:
# Lowercase
df_train['Text_data'] = df_train['Text_data'].apply(lambda x: x.lower())

In [8]:
# Dictionary of English contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have","i'll": "i will",
                     "i'll've": "i will have","i'm": "i am","I've": "i have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expand contractions
df_train['Text_data']=df_train['Text_data'].apply(lambda x:expand_contractions(x))

In [9]:
# Remove punctuation
df_train['Text_data'] = df_train['Text_data'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
# Remove extra spaces
df_train['Text_data'] = df_train['Text_data'].apply(lambda x: re.sub(' +',' ',x))

In [10]:
# Check emoji
emoji_result = df_train['Text_data'].apply(lambda x: demoji.findall(x))
for item in emoji_result.tolist():
    if  item:
        print(item)

{'❤️': 'red heart'}
{'❤️': 'red heart'}
{'❤️': 'red heart'}
{'❤️': 'red heart'}
{'❤️': 'red heart'}
{'❤️': 'red heart'}
{'🦑': 'squid'}
{'😭': 'loudly crying face', '🥺': 'pleading face'}
{'❤️': 'red heart'}
{'😎': 'smiling face with sunglasses'}
{'❤️': 'red heart'}
{'🦑': 'squid'}
{'😭': 'loudly crying face', '🥺': 'pleading face'}
{'❤️': 'red heart'}
{'😎': 'smiling face with sunglasses'}
{'🙃': 'upside-down face'}
{'❤️': 'red heart'}
{'🦑': 'squid'}
{'😭': 'loudly crying face', '🥺': 'pleading face'}
{'❤️': 'red heart'}
{'😎': 'smiling face with sunglasses'}
{'🙃': 'upside-down face'}
{'❤️': 'red heart'}
{'🦑': 'squid'}
{'😭': 'loudly crying face', '🥺': 'pleading face'}
{'❤️': 'red heart'}
{'😎': 'smiling face with sunglasses'}
{'🙃': 'upside-down face'}
{'🥰': 'smiling face with hearts'}
{'❤️': 'red heart'}
{'🦑': 'squid'}
{'😭': 'loudly crying face', '🥺': 'pleading face'}
{'❤️': 'red heart'}
{'😎': 'smiling face with sunglasses'}
{'🙃': 'upside-down face'}
{'😅': 'grinning face with sweat'}
{'❤️': 'red h

In [11]:
# Transform emoji
df_train['Text_data'] = df_train['Text_data'].astype(str).apply(lambda x: emoji.demojize(x, delimiters=(' ', ' ')))

In [12]:
# Check URL
URL_result = df_train['Text_data'].apply(lambda x: re.findall(r'http', x))
cnt = 0
for item in URL_result.tolist():
    if  item:
        cnt += 1
cnt

72

In [13]:
# Remove URL
df_train['Text_data'] = df_train['Text_data'].astype(str).apply(lambda x: re.sub(r'http\S+', '', x))

In [14]:
df_train.head()

,PID,Text_data,Label
0,train_pid_1,waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,my new years resolution i am gonna get my ass ...,moderate
2,train_pid_3,new year somone else feeling like 2020 will be...,moderate
3,train_pid_4,my story i guess hi im from germany and my eng...,moderate
4,train_pid_5,sat in the dark and cried myself going into th...,moderate


In [15]:
df_train.to_csv('train_preprocess.csv', index = False)
# df_train.to_csv('valid_preprocess.csv', index = False)